In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from pathlib import Path
from sklearn import metrics
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import torchvision

from datetime import datetime
import cv2
from collections import OrderedDict

In [2]:
import pickle

In [3]:
PATH = Path("/data2/yinterian/multi-task-romain")

In [4]:
with open(PATH/"obs_df_train.pickle", 'rb') as f:
    train = pickle.load(f)

In [5]:
with open(PATH/"obs_df_valid.pickle", 'rb') as f:
    valid = pickle.load(f)

In [6]:
train.head()

,subject_id,gender,age,sapsi_first,sofa_first,care_unit,amine,sedation,ventilation,period,series,y_event,y_mean_abpm,is_val
0,10013,1,87,15,8,0,1,0,0,10,"[[85.0, 89.1, 106.8, 31.5, 52.2, 1.0], [84.9, ...",1,62.465,0
1,10013,1,87,15,8,0,1,0,0,11,"[[85.9, 88.3, 123.2, 38.6, 63.2, 1.0], [85.5, ...",1,56.505,0
2,10013,1,87,15,8,0,1,0,0,12,"[[85.6, 93.6, 108.4, 34.7, 55.9, 1.0], [86.0, ...",1,60.52,0
3,10013,1,87,15,8,0,1,0,0,15,"[[86.9, 0.0, 95.3, 30.0, 48.5, 1.0], [87.4, 0....",1,55.23,0
4,10013,1,87,15,8,0,1,0,0,16,"[[87.3, 90.6, 105.9, 36.1, 55.9, 1.0], [86.8, ...",1,57.965,0


In [7]:
valid.head()

,subject_id,gender,age,sapsi_first,sofa_first,care_unit,amine,sedation,ventilation,period,series,y_event,y_mean_abpm,is_val
221,10209,0,35,16,7,1,1,1,1,10,"[[101.1, 100.0, 149.3, 94.5, 115.7, 0.0], [101...",0,107.575,1
222,10209,0,35,16,7,1,1,1,1,102,"[[89.0, 100.0, 133.6, 69.2, 88.2, 0.0], [89.1,...",0,90.04,1
223,10209,0,35,16,7,1,1,1,1,103,"[[89.8, 97.9, 141.9, 71.2, 89.9, 0.0], [89.8, ...",0,96.515,1
224,10209,0,35,16,7,1,1,1,1,104,"[[89.4, 99.0, 159.3, 77.4, 98.1, 0.0], [89.1, ...",0,97.355,1
225,10209,0,35,16,7,1,1,1,1,106,"[[87.0, 99.0, 166.9, 78.4, 101.6, 0.0], [90.6,...",0,102.24,1


In [8]:
train.shape, valid.shape

((32256, 14), (7332, 14))

## Dataset

In [8]:
def get_mean_std_series(train):
    ss = np.concatenate(train.series.values)
    ss = ss.reshape(-1,6)
    return ss.mean(axis=0), ss.std(axis=0)

In [9]:
def get_mean_std_static(train):
    res = {}
    for name in ["age", "sapsi_first", "sofa_first"]:
        values = train[name].values
        res[name] = (values.mean(), values.std())
    res["series"] = get_mean_std_series(train)
    return res

In [10]:
norm_dict = get_mean_std_static(train)
norm_dict 

{'age': (67, 16.152217089222916),
 'sapsi_first': (16, 4.968204119744002),
 'sofa_first': (8, 4.153183597602336),
 'series': (array([ 85.78689748,  90.61373362, 118.25838821,  57.44024058,
          81.28476211,   0.17137897]),
  array([16.91317423, 24.11194065, 31.60648773, 17.01183444, 20.30982141,
          0.37683978]))}

In [11]:
class MultiTask(Dataset):
    def __init__(self, df, norm_dict):
        """
        Args:
            df: dataframe with data
            norm_dict: mean and std of all variables to normalize
            
        """
        self.norm_dict = norm_dict
        self.df = df
        self.names = ["age", "sapsi_first", "sofa_first"] ## needs normalization
        self.names_binary = ["gender", "amine", "sedation", "ventilation"]

    def __getitem__(self, index):
        row = self.df.iloc[index,:]
        x_series = (row.series - self.norm_dict["series"][0])/self.norm_dict["series"][1]
        x_cont = [(row[name]-self.norm_dict[name][0])/self.norm_dict[name][1] for name in self.names]
        x_binary = [row[name] for name in self.names_binary]
        x_cat = row["care_unit"]
        x_cont = np.array(x_cont + x_binary)
        return x_series, x_cont, x_cat, row["y_event"], row["y_mean_abpm"]

    def __len__(self):
        return self.df.shape[0]

In [12]:
train_ds = MultiTask(train, norm_dict)
valid_ds = MultiTask(valid, norm_dict)

In [17]:
batch_size = 5
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size)

In [18]:
x_series, x_cont, x_cat, y1, y2 = next(iter(train_dl))

In [19]:
x_series.shape

torch.Size([5, 60, 6])

In [20]:
x_cont.shape

torch.Size([5, 7])

In [22]:
gru = nn.GRU(6, 11, bidirectional=True, batch_first=True).cuda()

In [23]:
hts, ht = gru(x_series.float().cuda())
ht.shape

torch.Size([2, 5, 11])

In [24]:
ht[0].shape, ht[1].shape

(torch.Size([5, 11]), torch.Size([5, 11]))

In [90]:
hts.shape

torch.Size([5, 60, 11])

In [102]:
hidden_size = 100
embedding = nn.Embedding(3, 1).cuda()
gru = nn.GRU(6, hidden_size, bidirectional=True, batch_first=True).cuda()
num = hidden_size + 10 + 1
linear1 = nn.Linear(7, 10).cuda()
out1 = nn.Linear(num, 1).cuda()
out2 = nn.Linear(num, 1).cuda()
bn1 = nn.BatchNorm1d(10).cuda()

In [105]:
x_series, x_cont, x_cat, y1, y2 = next(iter(train_dl))
x_series = x_series.float().cuda()
x_cont = x_cont.float().cuda()
x_cat = x_cat.long().cuda()
y1 = y1.float().cuda()
y2 = y2.float().cuda()

In [104]:
_, ht = gru(x_series)

## Model

In [13]:
def save_model(m, p): torch.save(m.state_dict(), p)
    
def load_model(m, p): m.load_state_dict(torch.load(p))

In [63]:
class EventModel(nn.Module):
    def __init__(self, hidden_size=100):
        super(EventModel, self).__init__()
        self.embedding = nn.Embedding(3, 1)
        self.gru = nn.GRU(6, hidden_size, batch_first=True)
        self.num = hidden_size + 10 + 1
        self.linear1 = nn.Linear(7, 10)
        self.out1 = nn.Linear(self.num, 1)
        self.out2 = nn.Linear(self.num, 1)
        self.bn1 = nn.BatchNorm1d(10)

    def forward(self, x_series, x_cont, x_cat):
        _, ht = self.gru(x_series)
        x_cat = self.embedding(x_cat)
        x_cont = self.bn1(F.relu(self.linear1(x_cont))) 
        x = torch.cat((ht[-1], x_cat, x_cont), 1)
        return self.out1(x), self.out2(x)

In [32]:
class EventModel2(nn.Module):
    def __init__(self, hidden_size=50):
        super(EventModel2, self).__init__()
        self.embedding = nn.Embedding(3, 1)
        self.gru = nn.GRU(6, hidden_size, bidirectional=True, batch_first=True)
        self.num = 2*hidden_size + 7 + 1
        self.linear1 = nn.Linear(7, 7)
        self.out1 = nn.Linear(self.num, 1)
        self.out2 = nn.Linear(self.num, 1)
        self.bn1 = nn.BatchNorm1d(7)
        
    def forward(self, x_series, x_cont, x_cat):
        _, ht = self.gru(x_series)
        x_cat = self.embedding(x_cat)
        x_cont = self.bn1(F.relu(self.linear1(x_cont))) + x_cont
        x = torch.cat((ht[0], ht[1], x_cat, x_cont), 1)
        return self.out1(x), self.out2(x)

In [41]:
class EventModel3(nn.Module):
    def __init__(self, hidden_size=100):
        super(EventModel3, self).__init__()
        self.embedding = nn.Embedding(3, 1)
        self.gru = nn.GRU(6, hidden_size, batch_first=True)
        self.num = hidden_size + 7 + 1
        self.linear1 = nn.Linear(7, 7)
        self.out1 = nn.Linear(self.num, 1)
        self.out2 = nn.Linear(self.num, 1)
        self.bn1 = nn.BatchNorm1d(7)
        
    def forward(self, x_series, x_cont, x_cat):
        _, ht = self.gru(x_series)
        x_cat = self.embedding(x_cat)
        x_cont = self.bn1(F.relu(self.linear1(x_cont))) + x_cont
        x = torch.cat((ht[-1], x_cat, x_cont), 1)
        return self.out1(x), self.out2(x)

In [28]:
def val_metrics(model, valid_dl):
    model.eval()
    total = 0
    sum_loss = 0
    y_hat1 = []
    ys1 = []
    y_hat2 = []
    ys2 = []
    for x_series, x_cont, x_cat, y1, y2 in valid_dl:
        batch = y1.shape[0]
        x_series = x_series.float().cuda()
        x_cont = x_cont.float().cuda()
        x_cat = x_cat.long().cuda()
        y1 = y1.float().cuda()
        y2 = y2.float().cuda()
        out1, out2 = model(x_series, x_cont, x_cat)
        log_loss = F.binary_cross_entropy_with_logits(out1, y1.unsqueeze(-1))
        mse_loss = F.mse_loss(out2, y2.unsqueeze(-1))
        loss = log_loss + mse_loss
        sum_loss += batch*(loss.item())
        total += batch
        y_hat1.append(out1.view(-1).detach().cpu().numpy())
        ys1.append(y1.view(-1).cpu().numpy())
        y_hat2.append(out2.view(-1).detach().cpu().numpy())
        ys2.append(y2.view(-1).cpu().numpy())
    
    y_hat1 = np.concatenate(y_hat1)
    y_hat2 = np.concatenate(y_hat2)
    ys1 = np.concatenate(ys1)
    ys2 = np.concatenate(ys2)
    r2 = metrics.r2_score(ys2, y_hat2)
    auc = metrics.roc_auc_score(ys1, y_hat1)
    return sum_loss/total, auc, r2

In [29]:
def train_epochs(model, train_dl, optimizer, lr=1e-3, epochs = 30):
    for i in range(epochs):
        sum_loss1 = 0
        sum_loss2 = 0
        total = 0
        for x_series, x_cont, x_cat, y1, y2 in train_dl:
            model.train()
            x_series = x_series.float().cuda()
            x_cont = x_cont.float().cuda()
            x_cat = x_cat.long().cuda()
            y1 = y1.float().cuda()
            y2 = y2.float().cuda()
            out1, out2 = model(x_series, x_cont, x_cat)
            log_loss = F.binary_cross_entropy_with_logits(out1, y1.unsqueeze(-1))
            mse_loss = F.mse_loss(out2, y2.unsqueeze(-1))
            loss = log_loss + mse_loss
            sum_loss1 += len(y1) * log_loss.item()
            sum_loss2 += len(y1) * mse_loss.item()
            total += len(y1)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            #val_loss, val_r2 = val_metrics2(model, val_dl2)  # val_dl
            #  Valid loss: {:.3f} {:.3f} \t Valid R2:  {:.3f}
        if i % 1 == 0:
            val_loss, val_auc, val_r2 = val_metrics(model, valid_dl)
            print("\tTrain loss: {:.3f} {:.3f} valid loss: {:.3f} valid auc {:.3f} valid r2 {:.3f}".format(
                sum_loss1/total, sum_loss2/total, val_loss, val_auc, val_r2))

In [181]:
batch_size = 5000
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size)

In [64]:
model = EventModel().cuda()

In [65]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.05, weight_decay=1e-5)

In [66]:
train_epochs(model, train_dl, optimizer, epochs=15)

	Train loss: 0.457 5476.242 valid loss: 3371.622 valid auc 0.984 valid r2 -9.333
	Train loss: 0.163 2287.411 valid loss: 866.005 valid auc 0.990 valid r2 -1.654
	Train loss: 0.066 455.009 valid loss: 232.331 valid auc 0.999 valid r2 0.288
	Train loss: 0.023 269.016 valid loss: 381.748 valid auc 0.997 valid r2 -0.170
	Train loss: 0.052 249.567 valid loss: 190.040 valid auc 0.988 valid r2 0.418
	Train loss: 0.120 146.674 valid loss: 156.164 valid auc 0.976 valid r2 0.522
	Train loss: 0.129 144.540 valid loss: 152.921 valid auc 0.985 valid r2 0.532
	Train loss: 0.070 132.529 valid loss: 138.863 valid auc 0.995 valid r2 0.575
	Train loss: 0.039 123.282 valid loss: 135.208 valid auc 0.996 valid r2 0.586
	Train loss: 0.033 119.760 valid loss: 129.378 valid auc 0.995 valid r2 0.604
	Train loss: 0.029 115.332 valid loss: 127.812 valid auc 0.995 valid r2 0.608
	Train loss: 0.025 113.677 valid loss: 124.155 valid auc 0.996 valid r2 0.620
	Train loss: 0.021 110.993 valid loss: 122.927 valid auc 0

In [67]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.03, weight_decay=1e-5)
train_epochs(model, train_dl, optimizer, epochs=20)

	Train loss: 0.018 106.256 valid loss: 114.914 valid auc 0.999 valid r2 0.648
	Train loss: 0.009 103.002 valid loss: 110.028 valid auc 0.999 valid r2 0.663
	Train loss: 0.005 97.049 valid loss: 106.884 valid auc 1.000 valid r2 0.672
	Train loss: 0.002 93.201 valid loss: 103.082 valid auc 1.000 valid r2 0.684
	Train loss: 0.001 89.498 valid loss: 104.673 valid auc 1.000 valid r2 0.679
	Train loss: 0.001 91.438 valid loss: 100.413 valid auc 1.000 valid r2 0.692
	Train loss: 0.000 86.177 valid loss: 96.543 valid auc 1.000 valid r2 0.704
	Train loss: 0.000 85.200 valid loss: 96.200 valid auc 1.000 valid r2 0.705
	Train loss: 0.000 82.420 valid loss: 95.452 valid auc 1.000 valid r2 0.707
	Train loss: 0.000 81.668 valid loss: 96.489 valid auc 1.000 valid r2 0.704
	Train loss: 0.000 81.975 valid loss: 95.094 valid auc 1.000 valid r2 0.709
	Train loss: 0.000 81.277 valid loss: 94.531 valid auc 1.000 valid r2 0.710
	Train loss: 0.000 80.508 valid loss: 93.134 valid auc 1.000 valid r2 0.715
	Tra

In [ ]:
train_epochs(model, train_dl, optimizer, epochs=10)

In [50]:
path = "{0}/models/model_acc_1_r2_{1:.0f}.pth".format(PATH, 100*0.71) 
save_model(model, path)

In [213]:
# new try
model = ReviewModel().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05, weight_decay=1e-5)
train_epochs(model, train_dl, optimizer, epochs=10)

	Train loss: 0.510 5661.317 valid loss: 3645.168 valid auc 0.979 valid r2 -10.171
	Train loss: 0.288 2402.384 valid loss: 1024.066 valid auc 0.996 valid r2 -2.138
	Train loss: 0.071 445.740 valid loss: 204.697 valid auc 0.999 valid r2 0.373
	Train loss: 0.040 295.383 valid loss: 353.198 valid auc 0.995 valid r2 -0.082
	Train loss: 0.123 286.915 valid loss: 185.472 valid auc 0.978 valid r2 0.432
	Train loss: 0.198 150.172 valid loss: 162.997 valid auc 0.976 valid r2 0.501
	Train loss: 0.138 147.899 valid loss: 158.680 valid auc 0.991 valid r2 0.514
	Train loss: 0.064 132.395 valid loss: 136.314 valid auc 0.998 valid r2 0.582
	Train loss: 0.030 118.771 valid loss: 128.493 valid auc 0.999 valid r2 0.606
	Train loss: 0.025 112.281 valid loss: 121.999 valid auc 0.999 valid r2 0.626


In [214]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.03, weight_decay=1e-5)
train_epochs(model, train_dl, optimizer, epochs=10)

	Train loss: 0.030 107.457 valid loss: 114.954 valid auc 0.999 valid r2 0.648
	Train loss: 0.016 99.500 valid loss: 108.701 valid auc 0.999 valid r2 0.667
	Train loss: 0.012 93.712 valid loss: 104.976 valid auc 0.999 valid r2 0.678
	Train loss: 0.009 91.632 valid loss: 103.072 valid auc 1.000 valid r2 0.684
	Train loss: 0.005 86.938 valid loss: 98.072 valid auc 1.000 valid r2 0.699
	Train loss: 0.003 84.544 valid loss: 96.656 valid auc 1.000 valid r2 0.704
	Train loss: 0.002 84.164 valid loss: 95.917 valid auc 1.000 valid r2 0.706
	Train loss: 0.002 81.947 valid loss: 94.791 valid auc 1.000 valid r2 0.709
	Train loss: 0.002 80.682 valid loss: 93.909 valid auc 1.000 valid r2 0.712
	Train loss: 0.002 79.567 valid loss: 93.170 valid auc 1.000 valid r2 0.714


In [215]:
path = "{0}/models/model_acc_1_r2_{1:.0f}.pth".format(PATH, 100*0.71) 
save_model(model, path)

In [219]:
# new try
model = ReviewModel().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05, weight_decay=3e-5)
train_epochs(model, train_dl, optimizer, epochs=15)

	Train loss: 0.515 5614.094 valid loss: 3347.934 valid auc 0.963 valid r2 -9.260
	Train loss: 0.296 2136.751 valid loss: 786.024 valid auc 0.994 valid r2 -1.409
	Train loss: 0.081 370.051 valid loss: 245.979 valid auc 0.999 valid r2 0.246
	Train loss: 0.039 311.871 valid loss: 268.143 valid auc 0.997 valid r2 0.178
	Train loss: 0.073 196.453 valid loss: 160.368 valid auc 0.987 valid r2 0.509
	Train loss: 0.115 136.225 valid loss: 150.191 valid auc 0.982 valid r2 0.540
	Train loss: 0.105 127.142 valid loss: 134.692 valid auc 0.992 valid r2 0.587
	Train loss: 0.061 117.468 valid loss: 127.828 valid auc 0.994 valid r2 0.608
	Train loss: 0.041 112.376 valid loss: 123.768 valid auc 0.996 valid r2 0.621
	Train loss: 0.030 107.708 valid loss: 122.921 valid auc 0.996 valid r2 0.623
	Train loss: 0.023 105.248 valid loss: 120.223 valid auc 0.997 valid r2 0.632
	Train loss: 0.019 102.736 valid loss: 114.898 valid auc 0.998 valid r2 0.648
	Train loss: 0.016 99.834 valid loss: 112.167 valid auc 0.9

In [220]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.03, weight_decay=3e-5)
train_epochs(model, train_dl, optimizer, epochs=10)

	Train loss: 0.017 115.906 valid loss: 115.094 valid auc 0.998 valid r2 0.647
	Train loss: 0.011 99.338 valid loss: 110.153 valid auc 0.999 valid r2 0.662
	Train loss: 0.008 93.230 valid loss: 105.050 valid auc 0.999 valid r2 0.678
	Train loss: 0.007 89.889 valid loss: 102.571 valid auc 1.000 valid r2 0.686
	Train loss: 0.003 87.512 valid loss: 101.051 valid auc 1.000 valid r2 0.690
	Train loss: 0.002 86.400 valid loss: 99.636 valid auc 1.000 valid r2 0.695
	Train loss: 0.001 84.975 valid loss: 99.361 valid auc 1.000 valid r2 0.695
	Train loss: 0.001 82.448 valid loss: 97.378 valid auc 1.000 valid r2 0.702
	Train loss: 0.001 81.799 valid loss: 97.014 valid auc 1.000 valid r2 0.703
	Train loss: 0.001 81.172 valid loss: 96.950 valid auc 1.000 valid r2 0.703


In [218]:
train_epochs(model, train_dl, optimizer, epochs=10)

	Train loss: 0.003 80.319 valid loss: 97.518 valid auc 1.000 valid r2 0.701
	Train loss: 0.002 79.188 valid loss: 95.136 valid auc 1.000 valid r2 0.708
	Train loss: 0.002 78.780 valid loss: 97.277 valid auc 1.000 valid r2 0.702
	Train loss: 0.002 78.105 valid loss: 95.397 valid auc 1.000 valid r2 0.708
	Train loss: 0.002 77.212 valid loss: 95.913 valid auc 1.000 valid r2 0.706
	Train loss: 0.001 77.310 valid loss: 94.746 valid auc 1.000 valid r2 0.710
	Train loss: 0.001 76.886 valid loss: 96.049 valid auc 1.000 valid r2 0.706
	Train loss: 0.001 75.821 valid loss: 95.151 valid auc 1.000 valid r2 0.708
	Train loss: 0.002 75.586 valid loss: 96.667 valid auc 1.000 valid r2 0.704
	Train loss: 0.002 76.552 valid loss: 105.541 valid auc 1.000 valid r2 0.677


## bidirectional model

In [30]:
batch_size = 5000
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size)

In [34]:
model = EventModel2().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05, weight_decay=3e-5)
train_epochs(model, train_dl, optimizer, epochs=10)

	Train loss: 0.512 5396.758 valid loss: 2867.637 valid auc 0.748 valid r2 -7.788
	Train loss: 0.476 1736.363 valid loss: 561.296 valid auc 0.888 valid r2 -0.719
	Train loss: 0.329 332.075 valid loss: 360.509 valid auc 0.972 valid r2 -0.104
	Train loss: 0.247 376.701 valid loss: 405.927 valid auc 0.961 valid r2 -0.243
	Train loss: 0.264 299.431 valid loss: 256.057 valid auc 0.910 valid r2 0.216
	Train loss: 0.311 193.591 valid loss: 210.667 valid auc 0.900 valid r2 0.355
	Train loss: 0.309 178.996 valid loss: 195.992 valid auc 0.954 valid r2 0.400
	Train loss: 0.273 162.775 valid loss: 178.908 valid auc 0.978 valid r2 0.452
	Train loss: 0.222 154.556 valid loss: 171.005 valid auc 0.986 valid r2 0.476
	Train loss: 0.176 148.443 valid loss: 160.542 valid auc 0.990 valid r2 0.508


In [35]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.03, weight_decay=3e-5)
train_epochs(model, train_dl, optimizer, epochs=10)

	Train loss: 0.205 138.009 valid loss: 142.054 valid auc 0.992 valid r2 0.565
	Train loss: 0.123 119.375 valid loss: 121.359 valid auc 0.994 valid r2 0.628
	Train loss: 0.092 104.652 valid loss: 112.241 valid auc 0.994 valid r2 0.656
	Train loss: 0.071 96.723 valid loss: 106.069 valid auc 0.995 valid r2 0.675
	Train loss: 0.061 91.812 valid loss: 102.572 valid auc 0.996 valid r2 0.686
	Train loss: 0.059 88.423 valid loss: 100.487 valid auc 0.997 valid r2 0.692
	Train loss: 0.054 86.139 valid loss: 99.896 valid auc 0.997 valid r2 0.694
	Train loss: 0.046 84.702 valid loss: 98.534 valid auc 0.997 valid r2 0.698
	Train loss: 0.039 83.165 valid loss: 96.876 valid auc 0.998 valid r2 0.703
	Train loss: 0.034 83.170 valid loss: 95.775 valid auc 0.998 valid r2 0.707


In [36]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.03, weight_decay=3e-5)
train_epochs(model, train_dl, optimizer, epochs=10)

	Train loss: 0.041 95.387 valid loss: 101.649 valid auc 0.998 valid r2 0.689
	Train loss: 0.027 85.841 valid loss: 97.907 valid auc 0.998 valid r2 0.700
	Train loss: 0.018 82.423 valid loss: 98.105 valid auc 0.999 valid r2 0.699
	Train loss: 0.013 80.887 valid loss: 94.609 valid auc 0.999 valid r2 0.710
	Train loss: 0.011 79.400 valid loss: 94.408 valid auc 0.999 valid r2 0.711
	Train loss: 0.009 78.400 valid loss: 95.185 valid auc 0.999 valid r2 0.708
	Train loss: 0.009 77.394 valid loss: 93.863 valid auc 0.999 valid r2 0.712
	Train loss: 0.009 76.556 valid loss: 92.947 valid auc 0.999 valid r2 0.715
	Train loss: 0.009 76.214 valid loss: 93.462 valid auc 0.999 valid r2 0.714
	Train loss: 0.008 75.490 valid loss: 92.902 valid auc 0.999 valid r2 0.715


In [37]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.03, weight_decay=3e-5)
train_epochs(model, train_dl, optimizer, epochs=10)

	Train loss: 0.011 83.232 valid loss: 101.910 valid auc 0.999 valid r2 0.688
	Train loss: 0.009 80.226 valid loss: 95.778 valid auc 0.999 valid r2 0.706
	Train loss: 0.008 77.625 valid loss: 94.975 valid auc 0.999 valid r2 0.709
	Train loss: 0.006 75.138 valid loss: 93.962 valid auc 0.998 valid r2 0.712
	Train loss: 0.005 74.046 valid loss: 94.162 valid auc 0.999 valid r2 0.711
	Train loss: 0.005 72.985 valid loss: 93.117 valid auc 0.999 valid r2 0.715
	Train loss: 0.005 74.338 valid loss: 100.073 valid auc 0.999 valid r2 0.693
	Train loss: 0.005 74.674 valid loss: 95.869 valid auc 0.999 valid r2 0.706
	Train loss: 0.005 74.494 valid loss: 96.297 valid auc 0.999 valid r2 0.705
	Train loss: 0.004 73.423 valid loss: 95.504 valid auc 0.999 valid r2 0.707


In [39]:
### Model 2

In [42]:
model = EventModel3().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05, weight_decay=3e-5)
train_epochs(model, train_dl, optimizer, epochs=15)

	Train loss: 0.448 5422.415 valid loss: 3257.136 valid auc 0.965 valid r2 -8.982
	Train loss: 0.188 2126.582 valid loss: 727.760 valid auc 0.982 valid r2 -1.230
	Train loss: 0.101 382.944 valid loss: 262.469 valid auc 0.999 valid r2 0.196
	Train loss: 0.050 286.992 valid loss: 394.148 valid auc 0.996 valid r2 -0.208
	Train loss: 0.093 283.393 valid loss: 216.445 valid auc 0.985 valid r2 0.337
	Train loss: 0.175 159.576 valid loss: 159.808 valid auc 0.971 valid r2 0.511
	Train loss: 0.194 147.195 valid loss: 161.487 valid auc 0.980 valid r2 0.506
	Train loss: 0.126 140.781 valid loss: 146.122 valid auc 0.993 valid r2 0.552
	Train loss: 0.062 128.764 valid loss: 138.021 valid auc 0.997 valid r2 0.577
	Train loss: 0.042 124.727 valid loss: 132.152 valid auc 0.998 valid r2 0.595
	Train loss: 0.032 117.261 valid loss: 126.010 valid auc 0.998 valid r2 0.614
	Train loss: 0.025 112.174 valid loss: 120.545 valid auc 0.997 valid r2 0.631
	Train loss: 0.024 108.676 valid loss: 116.961 valid auc 0

In [44]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.03, weight_decay=3e-5)
train_epochs(model, train_dl, optimizer, epochs=15)

	Train loss: 0.025 101.943 valid loss: 111.617 valid auc 0.998 valid r2 0.658
	Train loss: 0.018 97.338 valid loss: 107.893 valid auc 0.998 valid r2 0.669
	Train loss: 0.015 93.232 valid loss: 104.515 valid auc 0.999 valid r2 0.680
	Train loss: 0.009 89.917 valid loss: 100.391 valid auc 0.999 valid r2 0.692
	Train loss: 0.007 87.176 valid loss: 100.372 valid auc 0.999 valid r2 0.692
	Train loss: 0.005 85.816 valid loss: 98.958 valid auc 1.000 valid r2 0.697
	Train loss: 0.005 84.202 valid loss: 97.510 valid auc 1.000 valid r2 0.701
	Train loss: 0.003 82.915 valid loss: 97.885 valid auc 1.000 valid r2 0.700
	Train loss: 0.002 82.477 valid loss: 98.357 valid auc 1.000 valid r2 0.699
	Train loss: 0.002 82.003 valid loss: 98.049 valid auc 1.000 valid r2 0.699
	Train loss: 0.002 80.348 valid loss: 95.627 valid auc 1.000 valid r2 0.707
	Train loss: 0.002 80.548 valid loss: 94.659 valid auc 1.000 valid r2 0.710
	Train loss: 0.002 79.668 valid loss: 94.605 valid auc 1.000 valid r2 0.710
	Train